# Gen AI Demo

__Import OpenAI key__

In [7]:
from secret_key import openai_api_key

__Import OpenAI, ChatOpenAI, PromptTemplate, ChatPromptTemplate__

In [4]:
%pip install -qU langchain-openai
%pip install -qU langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

__Use LangChain OpenAI, LangChain ChatOpenAI__

In [28]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key=openai_api_key, temperature=0.7)
llm.invoke("How far is the moon away from earth in miles ?")

'\n\nThe average distance between the moon and Earth is approximately 238,855 miles.'

In [31]:
chat_llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0.7)
chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You are an expert in Python programming language"),
        HumanMessagePromptTemplate.from_template("Tell me about scikit learn in 2 sentences")
])
                                     
chain = chat_prompt_template | chat_llm

chain.invoke({})

AIMessage(content='Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data mining and data analysis. It includes various algorithms for classification, regression, clustering, dimensionality reduction, and more.', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 29, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-665aedf9-4150-444d-b291-d45d1f7a3783-0', usage_metadata={'input_tokens': 29, 'output_tokens': 43, 'total_tokens': 72})

__ChatPromptTemplate with dynamic values__

In [32]:
chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You are an expert in {main_topic}"),
        HumanMessagePromptTemplate.from_template("Tell me about {sub_topic} in 2 sentences")
])
                                     
chain = chat_prompt_template | chat_llm

chain.invoke({"main_topic": "Node.js", "sub_topic": "closures"})

AIMessage(content='In Node.js, closures are functions that remember the variables from the outer scope even after the outer function has finished executing. This allows the inner function to access and manipulate those variables, creating a private scope within the function.', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 26, 'total_tokens': 70}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bdd581f4-f805-47eb-a957-df1dd2534038-0', usage_metadata={'input_tokens': 26, 'output_tokens': 44, 'total_tokens': 70})

In [33]:
chain.invoke({"main_topic": "SAP Integration Suite", "sub_topic": "OData Adapter"})

AIMessage(content='The OData Adapter in SAP Integration Suite allows you to easily connect and integrate OData services with other systems and applications. It provides capabilities to consume and expose OData services, enabling seamless data exchange and interoperability within your integration landscape.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 29, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-61ba33c0-5dc2-4a09-b903-8e7dd3778edd-0', usage_metadata={'input_tokens': 29, 'output_tokens': 47, 'total_tokens': 76})

__Output Parsers - Format the output for APIs__

In [35]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({"main_topic": "Node.js", "sub_topic": "closures"})

'Closures in JavaScript allow functions to retain access to variables from their containing scope even after the parent function has finished executing. This enables functions to "remember" and manipulate those variables when called later on.'

In [37]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You are an English teacher"),
        HumanMessagePromptTemplate.from_template("Provide 5 syonyms for the following word {word}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({"word": "determination"})

# print(type(chain.invoke({"word": "determination"})))

<class 'list'>


In [45]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Employee(BaseModel):
    name: str = Field(description="Name of person")
    company: str = Field(description="Name of company")
    experience: int = Field(description="Number of years with company")

output_parser = JsonOutputParser(pydantic_object=Employee)

chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("Create a fictitious employee with the following name.  \nFormatting Instructions: {format_instructions}"),
        HumanMessagePromptTemplate.from_template("Employee name: {name}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({
    "name": "Milton",
    "format_instructions": output_parser.get_format_instructions()
    })

# print(type(chain.invoke({
#     "name": "Milton",
#     "format_instructions": output_parser.get_format_instructions()
#     })
# ))

{'name': 'Milton', 'company': 'Acme Corporation', 'experience': 5}